In [ ]:
from utils import func
from utils import network
from utils import visualization
from matplotlib_venn import venn3, venn3_circles
import anndata
import numpy as np
import pylab as plt

In [ ]:
%cd data/
immune_cell = anndata.read_h5ad('all_cells.h5ad')

In [ ]:
### Build a multiplex network (two layers) for filtered gene sets in GO-CC
curated_cell_label = "Classical monocytes"
tissue_ns = ['BLD', 'BMA',]
for name in tissue_ns:
    globals()[name] = np.load(f'{name}_{curated_cell_label}_GO_CC_immunesig.npy', allow_pickle = True).tolist()
gs_names = BLD.gene_set['GO-CC'].gs_names
filter_index = BLD.gene_set['GO-CC'].filter_index['kmeans']
gs_names = np.array(gs_names)[filter_index]

In [ ]:
multiplex_nw_GO_CC = network.multiplex_tissue_network(gene_set_collection_name = 'GO-CC', layer = [eval(tis) for tis in tissue_ns],\
                                                layer_ns = tissue_ns, method = 'kmeans', self_loop = True)
multiplex_nw_GO_CC.load_Jaccard(self_loop = True)
### Remove negative edges or those with weak weights
multiplex_nw_GO_CC.call_adj_filter_tr(q = None, tr = 0.2)

In [ ]:
multiplex_nw_GO_CC.multiplex_participation_coefficient()
P = multiplex_nw_GO_CC.multiplex_property['P']

In [ ]:
multiplex_nw_GO_CC.multiplex_page_rank()
multiplex_nw_GO_CC.Jaccard_page_rank()

In [ ]:
multiplex_nw_GO_CC.C_1()
C1 = multiplex_nw_GO_CC.multiplex_property['C1']

In [ ]:
jpg = multiplex_nw_GO_CC.multiplex_property['Jaccard_page_rank']
mpg = multiplex_nw_GO_CC.multiplex_property['multiplex_page_rank']
C1 = multiplex_nw_GO_CC.multiplex_property['C1']
for ns in ["mpg", "jpg", "C1","P"]:
    if ns == 'mpg':
        denominator = (1/len(mpg))
    elif ns == 'jpg':
        denominator = (1/len(jpg))
    else:
        denominator = 1        
    globals()[ns+'_dict'] = {gs_names[i]:eval(ns)[i]/denominator for i in range(len(gs_names))}

#### Gene set enrichment analysis: Classical monocytes in BMA vs BLD

In [ ]:
for tis in ['BLD', 'BMA']:
    globals()[tis] = immune_cell[(immune_cell.obs['Organ'] == tis)\
                                 *(immune_cell.obs['Manually_curated_celltype'] == curated_cell_label)]

In [ ]:
gsea_res, pd_data = func.gsea_tissue(BMA, BLD, exp_n = "BMA", con_n = "BLD", \
                                     gene_set_path = "c5.go.cc.v2023.1.Hs.symbols.gmt",\
                                    para_gsea = {'permutation_type': 'phenotype', 'permutation_num':2000, 'outdir':None,\
                            'method':'signal_to_noise', 'threads':60, 'seed':30})

In [ ]:
### Enriched gene sets with FDR adjusted q-values less than 0.05
Enriched_df = gsea_res.res2d.loc[gsea_res.res2d['FWER p-val'] <= 0.05,]

In [ ]:
df, ax = visualization.vis_gsea(Enriched_df, rank_label = "NES", top = 20, ax_title = 'Enriched gene sets in GO-CC',\
                              x_label = "NES (absolute)", \
                              rank_dict = None, gs_ns = gs_names,\
                              fig_size = (10,6), 
                                title_size = 16, anno_font_size = 14)

In [ ]:
ax.figure.savefig('GSEA/enriched_gs_NES.png', dpi = 300)

In [ ]:
df, ax = visualization.vis_gsea(Enriched_df, ax_title = 'Enriched gene sets in GO-CC',
                                rank_label = "multiplex PageRank", \
                                x_label = "multiplex PageRank (ratio)", \
                                top = 20,
                                rank_dict = mpg_dict, \
                                gs_ns = gs_names,\
                                fig_size = (10,6), 
                                title_size = 16, anno_font_size = 14)

In [ ]:
ax.figure.savefig('GSEA/enriched_gs_mpg.png', dpi = 300)

In [ ]:
df, ax = visualization.vis_gsea(Enriched_df, ax_title = 'Enriched gene sets in GO-CC',
                                rank_label = "JPG", \
                                x_label = "PageRank-Jaccard (ratio)", \
                                top = 20,
                                rank_dict = jpg_dict, \
                                gs_ns = gs_names,\
                                fig_size = (10,6), 
                                title_size = 16, anno_font_size = 14)

In [ ]:
ax.figure.savefig('GSEA/enriched_gs_jpg.png', dpi = 300)

In [ ]:
df, ax = visualization.vis_gsea(Enriched_df, ax_title = 'Enriched gene sets in GO-CC',\
                              rank_label = "C1_coefficient", colorbar = 'Spectral', x_label = "C1",
                                top = 20,\
                rank_dict = C1_dict,  gs_ns = gs_names,\
                             fig_size = (10,6), 
                            title_size = 16, anno_font_size = 14)

In [ ]:
ax.figure.savefig('GSEA/enriched_gs_C1.png', dpi = 300)

In [ ]:
df, ax = visualization.vis_gsea(Enriched_df, ax_title = 'Enriched gene sets in GO-CC',\
                              rank_label = "P", colorbar = 'viridis', x_label = "multiplex participation coefficient",
                                top = 20,\
                rank_dict = P_dict,  reverse = True, gs_ns = gs_names,\
                             fig_size = (10,6), 
                            title_size = 16, anno_font_size = 14)

In [ ]:
ax.figure.savefig('GSEA/enriched_gs_P.png', dpi = 300)

In [ ]:
plt.figure(figsize=(10,10))
plt.rcParams['font.size'] = '14'
plt.title("")
v = venn3([set(c1_lst), set(mpg_lst), set(jpg_lst)], ('C1 coefficient', 'Multiplex PageRank centrality',  'Centrality in Jaccard layer'))

Reference: Fang, Zhuoqing, Xinyuan Liu, and Gary Peltz. "GSEApy: a comprehensive package for performing gene set enrichment analysis in Python." Bioinformatics (2022).